In [24]:
# pip install transformers torch scikit-learn numpy sentence-transformers ipywidgets

In [25]:
from transformers import AutoTokenizer, AutoModel
import torch
import os
import numpy as np
from sentence_transformers import SentenceTransformer
import csv
import json
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, utility, DataType
import os
import pandas as pd
import re


# Select model
- dont forget to  change model hidden dim if you changed model.


In [26]:
model_name = "BAAI/bge-m3"
model_dim = 1024

In [27]:
bge_model = SentenceTransformer(model_name, trust_remote_code=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Admin\Documents\NLP_Sys\ThaiBoardGameRAG\thaiboardgameRAG\notebook\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [28]:
#use for query and insert doc fell free to fix if you use model with tokenizer
def generate_embedding(text):
    # Use the SentenceTransformer model to generate embeddings
    embeddings = bge_model.encode(text,convert_to_numpy=False)
    return embeddings

# connect to MILVUS database

In [29]:

MILVUS_HOST = 'localhost'
MILVUS_PORT = '19530'

connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)

In [30]:
# Function to initialize Milvus collection if it doesn't exist
def initialize_milvus_collection(document_name):
    # Check if collection exists
    if not utility.has_collection(document_name):
        # Create collection if it doesn't exist
        # You may need to adjust the schema based on your specific requirements

        fields = [
            FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
            # FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024),  # Adjust dim if needed
            FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=model_dim),  # Adjust dim if needed

            FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535)
        ]
        schema = CollectionSchema(fields, "Document embeddings for Information database")
        collection = Collection(document_name, schema)
        
        # Create an IVF_FLAT index for the embedding field
        index_params = {
            "metric_type": "L2",
            "index_type": "IVF_FLAT",
            "params": {"nlist": model_dim}
        }
        collection.create_index("embedding", index_params)
    else:
        collection = Collection(document_name)
    
    return collection

In [32]:
# Initialize Milvus collection
collection_name = 'BGE_m3_note_book'
collection = initialize_milvus_collection(collection_name)
collection.load()

In [33]:
def insert_data_to_milvus(text):
    """
    insert data into Milvus collection
    """
    # Generate embeddings for the text
    embedding = generate_embedding(text)

    # Prepare the entity to be inserted
    entity = {
        "text": text,
        "embedding": embedding
    }


    # Insert the entity into Milvus
    insert_result = collection.insert([entity])


    # Ensure the changes are immediately searchable
    collection.flush()

In [34]:
def read_and_index_docs(docs_dir='../docs'):
    """
    Read .txt files from the specified directory and index their contents.
    
    :param docs_dir: Directory containing the .txt files
    """
    indexed_files = 0
    non_indexed_files = 0
    
    selected_lines = ['Meta data', 'Content']
    for filename in os.listdir(docs_dir):
        if filename.endswith('.txt'):
            file_path = os.path.join(docs_dir, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    
                # Split content into chunks of max 1000 characters, including the title in each chunk
                chunks = []
                lines = content.split('\n')
                title = lines[0]  # Get the first line as the title
                current_chunk = title + '\n'  # Start each chunk with the title
                for line in lines[1:]:  # Skip the first line (title) in this loop
                    if any(line.startswith(prefix) for prefix in selected_lines):
                        if len(current_chunk) + len(line) + 1 <= 1000:  # +1 for newline
                            current_chunk += line + '\n'
                        else:
                            if current_chunk:
                                chunks.append(current_chunk.strip())
                            current_chunk = title + '\n' + line + '\n'  # Start a new chunk with the title
                if current_chunk:
                    chunks.append(current_chunk.strip())

                # Index each chunk separately
                file_indexed = True
                for chunk in chunks:
                    insert_data_to_milvus(chunk)
                
                if file_indexed:
                    indexed_files += 1
                    print(f"Finished indexing all chunks from {filename}")
                else:
                    non_indexed_files += 1
            
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                non_indexed_files += 1

    print(f"Indexing complete. Indexed files: {indexed_files}, Non-indexed files: {non_indexed_files}")

In [42]:
#incase you want to insert data from csv or json
read_and_index_docs()

Finished indexing all chunks from 7 Wonders (TH) 7 สิ่งมหัศจรรย์.txt
Finished indexing all chunks from 7 Wonders Architects (TH) 7 สิ่งมหัศจรรย์ ยอดสถาปนิก.txt
Finished indexing all chunks from 8 BIT BOX (TH) 8 บิทบ๊อกซ์.txt
Finished indexing all chunks from Art Society (TH) ศิลป์สโมสร.txt
Finished indexing all chunks from Bandido (THEN) แบนดิโด.txt
Finished indexing all chunks from Betakkuma  Fart & Furious ตดทะลุนรก.txt
Finished indexing all chunks from Bloodbound (TH) สงครามแวมไพร์.txt
Finished indexing all chunks from Bloody Inn (TH) โรงแรมสีเลือด.txt
Finished indexing all chunks from Bloody Inn Carnies (TH) โรงแรมสีเลือด ชาวคณะหรรษา.txt
Finished indexing all chunks from Book - Captive (TH).txt
Finished indexing all chunks from Book - Knight (TH).txt
Finished indexing all chunks from Book - Sherlock Holmes & Moriarty Associates (TH) เชอร์ล็อคโฮล์มส์ & มอริอาร์ตี้ พันธมิตรอันตราย.txt
Finished indexing all chunks from Book - Sherlock Holmes four investigations (TH).txt
Finished index

In [43]:
def completions(query): 
    """
    search for the most relevant documents in Milvus based on the query.   
    """        
    query_embedding = generate_embedding(query).numpy().flatten().tolist()
    # Prepare search parameters
    search_param = {
        "metric_type": "L2",
        "params": {"nprobe": 10},
    }
    # Step 2: Retrieve top-10 documents from Milvus
    search_results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_param,
        limit=10,
        output_fields=["id", "text", "embedding"],
        expr=None
    )
    
    # Extract document texts and embeddings
    retrieved_documents = []
    document_embeddings = []
    for hits in search_results:
        for hit in hits:
            retrieved_documents.append(hit.entity)
            # embedding = hit.entity.get('embedding')
            # if embedding is not None:
            #     document_embeddings.append(embedding)
                
    return retrieved_documents

In [44]:
completions('มีเกมบอร์ดที่ได้แรงบันดาลใจจากสูตรลับตำรับดันเจียนไหม')

[id: 457634866563864505, distance: 0.7608269453048706, entity: {'id': 457634866563864505, 'text': 'Title: Monster Eater Dungeon Meshi (TH) มอนสเตอร์อีทเตอร์ สูตรลับตำรับดันเจียน\nContent: ในเกมนี้ผู้เล่นจะได้เข้าไปสู่โลกของสูตรลับตำรับดันเจียน การ์ตูนดังจากญี่ปุ่นที่มีธีมเกี่ยวกับการสำรวจดันเจียนเพื่อเป้าหมายที่แตกต่างกันไปในเวอร์ชั่นบอร์ดเกม ผู้เล่นจะได้สวมบทบาทเป็นหนึ่งในหัวหน้าจากปาร์ตี้ ไลออส, ชูโร่, ทันซ์, คาบรู และ กองทัพคานาเรีย ที่มีเป้าหมายในการสำรวจดันเจียนให้ถึงชั้นล่างสุดและกำจัดบอสลงให้ได้ ในดันเจียนนั้นแบ่งออกเป็น 2 ชั้น คือชั้นบนและชั้นล่าง ในระหว่างทางของการสำรวจดันเจียน ผู้เล่นจะได้พบเจอกับมอนสเตอร์,ไอเทม,เหตุการณ์พิเศษมากมาย และหากมีผู้สำรวจดันเจียนชั้นบนได้ครบ 100% บอสประจำชั้น คิเมร่าฟาลิน จะปรากฎตัวออกมาเพื่อขวางทางการลงสู่ดันเจียนชั้นล่าง ผู้เล่นจะต้องปราบบอสลงให้ได้ และลงไปสำรวจชั้นล่างต่อให้ครบ 100% อีกครั้ง เพื่อให้บอสตัวสุดท้าย จอมเวทคลั่งทิสเซิล ปรากฎตัวและปราบลงให้จงได้ เพื่อเป็นการจบเกมคะแนนจากเกมนี้จะมาจาก ไอเทม, มอนสเตอร์ (ที่ปรุงอาหารสำเร็จ), มอนสเตอร์ระ

# Test

every thing like homework.

In [45]:
testRAG = pd.read_csv('testRAG.csv') 

In [46]:
def simpleCheckRAGQuery(query, title,top_k=10):
    retrieved_documents = completions(query)
    titles = []
    for idx, doc in enumerate(retrieved_documents):
        if(idx >= top_k):
            break
        match = re.search(r'Title:\s*(.*?)\n', doc.text)
        if match:
            extracted_title = match.group(1)
            if extracted_title == title:
                titles.append(idx+1)
    return titles

    

In [47]:
def testRAG(csv = 'testRAG.csv',top_k=3):
    testRAG = pd.read_csv(csv)
    total = len(testRAG)
    MRR = 0
    failures = 0
    
    for idx, row in testRAG.iterrows():
        text = row['text']
        file_name = row['file_name']
        result = simpleCheckRAGQuery(text, file_name,top_k)
        
        lowest_rank = result[0] if len(result) > 0 else 0
        
        MRR += 1/lowest_rank if lowest_rank > 0 else 0
        failures += 0 if  len(result) == 0 else 1
        
        print(f"{idx}. Query: {text}, File Name: {file_name}, Rank: {lowest_rank}, Failures: {len(result) == 0}")
        
        # if result == False:
        #     print(f"Incorrect: {text}, File Name: {file_name}")
    
    print(f"Total: {total}, MRR: {MRR/total:.4f}, Failures: {failures/total:.4f}")
            
    return MRR/total, failures/total
            
    # print(f"Total: {total}, Correct: {correct}, Accuracy: {correct / total * 100:.2f}%")

In [48]:
testRAG()

0. Query: มีบอร์ดเกมอะไรที่ดัดแปลงมาจากการ์ตูนญี่ปุ่นเกี่ยวกับการสำรวจดันเจียนบ้าง?, File Name: Monster Eater Dungeon Meshi (TH) มอนสเตอร์อีทเตอร์ สูตรลับตำรับดันเจียน, Rank: 1, Failures: False
1. Query: เกมบอร์ดไหนให้เราเล่นเป็นหัวหน้าปาร์ตี้อย่าง ไลออส หรือ คาบรู?, File Name: Monster Eater Dungeon Meshi (TH) มอนสเตอร์อีทเตอร์ สูตรลับตำรับดันเจียน, Rank: 0, Failures: True
2. Query: มีเกมบอร์ดที่เกี่ยวกับการปรุงอาหารจากมอนสเตอร์ในดันเจียนไหม?, File Name: Monster Eater Dungeon Meshi (TH) มอนสเตอร์อีทเตอร์ สูตรลับตำรับดันเจียน, Rank: 1, Failures: False
3. Query: เกมไหนที่ต้องสำรวจดันเจียนให้ครบ 100% ถึงจะเจอบอส?, File Name: Monster Eater Dungeon Meshi (TH) มอนสเตอร์อีทเตอร์ สูตรลับตำรับดันเจียน, Rank: 1, Failures: False
4. Query: อยากได้บอร์ดเกมที่มีระบบแบ่งดันเจียนเป็นสองชั้น มีเกมไหนบ้าง?, File Name: Monster Eater Dungeon Meshi (TH) มอนสเตอร์อีทเตอร์ สูตรลับตำรับดันเจียน, Rank: 2, Failures: False
5. Query: เกมอะไรที่จบลงเมื่อปราบจอมเวทคลั่งทิสเซิลได้?, File Name: Monster Eater Dungeon 

(0.7407407407407408, 0.8333333333333334)